# Time Series Forecasting with XGBoost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

df = pd.read_csv('AEP_hourly.csv')